# Luiata Data Science Take-home Assignment - Part II: Modeling

### Jade Yun | [LinkedIn](https://www.linkedin.com/in/jadeyun/) | [GitHub](https://github.com/yuyun2)

## Overview
This notebook contains code for:
- Baseline model (Random Forest)
- Fine tune Random Forest Model using GridSearchCV
- RF on variables with high importance
- Gradient Boosting model using LightGBM
- Fine tune LightGBM Model using GridSearchCV
- Emsemble
- Understand Feature Importance

**Precision-Recall AUC (PRAUC)** is selected as metric of model performance. 

At each step, model performance is evaluated based on validation set. 

Precision@20Recall and AUC are also calculated for reference only.

## Summary

In this assignment, machine learning models are built to predict patients' claim in the next year using patients medical history data.

Features are built from 3 perspectives to characterize patients medical status:
1. Patients' geographical informaiton (i.e. age, gender)
2. Patients' doctor visit behavior (e.g. # of visits, # of visits within time window, # of specific medical procedures, # of days since last/first visits)
3. Patients' lab results (e.g. # of lab tests, # of lab tests within time window, avg value of specific tests, value of last test)

In 2 and 3, apart from macro behaviors like total # of visits, total # of lab tests, etc, patient history on some certain important resource codes and lab tests are calculated as well. 

To understand the relationship between dabetes and each resource code/lab code, patient claim rate is calcualted for each resource code and lab code. For the top ranked resource code and lab code, features are calculated as described in 2 and 3, code and more detailed explanation can be found in the part I notebook. Claim rate itself is not used as feature to prevent data leakage.

Before modeling, the data set is split into 3 parts: train, validation and test:

- Train set is used to train model and tune hyper-parameters
- Validation set works as held-out data to evaluate model performance
- Test set contains the data needs to be predicted

Modeling proecess (Precision-Recall AUC (PRAUC) on validation set is reported at each step) is: 

1. Baseline Random Forest model (no hyper-parameter tuning), PRAUC: 0.174
2. Random Forest on feature subset (high importance features from step 1), PRAUC: 0.174, no improvement
3. Random Forest tuned with GridSearchCV, PRAUC: 0.183
4. LightGBM model (no hyper-parameter tuning), PRAUC: 0.188
5. LightGBM model tuned with GridSearchCV, PRAUC: 0.193
6. Ensemble (Average of tuned Random Forest and LightGBM), PRAUC: 0.193, no improvement

In the modeling process, to prevent overfitting, cross-validation is used on train set.

Fine-tuned LightGBM is selected as final model, the full dataset (train+validation) is used to train the final model and then to predict test set.

Lastly, feature importance from both Random Forest and Light GBM. The top 5 most important features are:

1. Age, this is in accordance with research by American Diabetes Association.

2. Average value of test loinc_4548-4, which tests the patients Hemoglobin level in blood.

3. Number of days since last resource code cpt_83036. This is also related with Hemoglobin level.

4. Number of resource code icd10_I10. The code is related with hypertension, which is the most common comorbid condition in diabetes.

5. Number of days since last doctor visit. A recent doctor visit might indicate a claim.

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from functools import reduce

from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

### Train/Validation/Test Split

In [2]:
lumiata = create_engine('sqlite:///lumiata.db') # connect to sqlite database 

In [3]:
# Basic patient and claim information
base = pd.read_sql_query('SELECT * FROM basic_info', lumiata) # query from base table

# features about resource
res_cnt = pd.read_sql_query('SELECT * FROM res_num_visit', lumiata) # query from base table
res_onehot = pd.read_sql_query('SELECT * FROM res_one_hot', lumiata)
res_days = pd.read_sql_query('SELECT * FROM res_days', lumiata)

In [5]:
# features about observation
ob_cnt = pd.read_sql_query('SELECT * FROM obs_num_labs', lumiata)
ob_onehot = pd.read_sql_query('SELECT * FROM ob_one_hot', lumiata)
ob_last = pd.read_sql_query('SELECT * FROM last_ob_one_hot', lumiata)

In [6]:
# join all the tables together
dfs = [base, res_cnt, res_onehot, res_days, ob_cnt, ob_onehot, ob_last]
master = reduce(lambda left,right: pd.merge(left,right,on='patient_id',how='left'), dfs)

In [7]:
# create age
master['age'] = (pd.to_datetime('2016-12-31') - pd.to_datetime(master['birthday']))/(12*np.timedelta64(1, 'M'))

In [8]:
def split_df(df):
    """split master data into train, validation and test set
    """
    df['claim'] = 0       # create a new column for target
    #df['random'] = np.random.random(size=len(df))  # insert a column of random number to analyze feature importance
    
    tmp, test = df.loc[df['split'] == 'train'], df.loc[df['split'] == 'test']     # assign train & test
    tmp = tmp.loc[(tmp['tag_dm2'].isnull()) | (tmp['tag_dm2'] >= '2017-01-01')]   # filter out 'prior' from train
    
    # label claim as 1 only if tag_dm2 is before 2017-12-31
    tmp['claim'] = np.where(tmp['tag_dm2'] <= '2017-12-31', 1, 0)   
    
    # split train into train and validation 0.8/0.2
    np.random.seed(0)
    msk = np.random.rand(len(tmp)) < 0.8
    train, valid = tmp[msk], tmp[~msk]
    
    print('observations in train/valid/test: {}/{}/{}'.format(train.shape[0], valid.shape[0], test.shape[0]))
    return train, valid, test

In [9]:
train, valid, test = split_df(master)

observations in train/valid/test: 58879/14641/31403


### Function to Evaluate Model Performance

In [83]:
def evaluate_model(clf, df, features, plot_apr=False, target_recall=0.2, target='claim', fill_na=-1):
    """ evaluate model performance, return:
        - predictions (probablity)
        - auc
        - precision-recall auc
        - precision @ target recall (default 0.2)
    """
    
    if type(clf) is list:   # if a list of clfs is provided, calculate the average of all clfs prediction
        pred = np.zeros(len(df))
        for c in clf:     
            pred += c.predict_proba(df[features].fillna(fill_na))[:,1]
        pred = pred/len(clf)
    else:
        pred = clf.predict_proba(df[features].fillna(fill_na))[:,1]
        
    auc  = roc_auc_score(df[target], pred)
    pr_auc = average_precision_score(df[target], pred)
    
    precision, recall, _ = precision_recall_curve(df[target], pred)
    idx = (np.abs(recall - target_recall)).argmin()
    pr_recall = precision[idx]    # precision at target recall
    
    if plot_apr:
        
        plt.step(recall, precision, color='b', alpha=0.2, where='post')
        plt.fill_between(recall, precision, step='post', alpha=0.2, color='b')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.ylim([0.0, 1.05])
        plt.xlim([0.0, 1.0])
        plt.title('Precision-Recall Curve: AP={0:0.2f}'.format(apr_auc))
    
    print("PRAUC: %.3f, Precision@%dRecall: %.3f, AUC: %.3f" %(pr_auc, int(target_recall*100), pr_recall, auc))
    
    return pred, pr_auc, pr_recall, auc

In [11]:
def avg_precision(labels, preds):
    """
    http://lightgbm.readthedocs.io/en/latest/_modules/lightgbm/sklearn.html
    self-defined eval metric for lightgbm
    f(labels: array, preds: array) -> name: string, value: array, is_higher_better: bool
    average precision (PRAUC)
    """
    return 'pr_auc', average_precision_score(labels, preds), True

In [12]:
def get_feature_importance(clf, features):
    """return ranked feature importance in pandas dataframe
    """ 
    imp = pd.DataFrame({'feature': features,
                        'importance': clf.feature_importances_}).sort_values('importance',ascending=False)
    return imp

### Baseline Model

Use random forest classifier as a base model

In [93]:
# dedine response and dependent variables 
target = 'claim'

base_cols = ['age','is_male']
res_cols = [v for v in list(res_cnt)+list(res_onehot)+list(res_days) if v != 'patient_id']
ob_cols = [v for v in list(ob_cnt)+list(ob_onehot)+list(ob_last) if v != 'patient_id']

var = base_cols + res_cols + ob_cols

In [94]:
# initialize a random forest
rfc = RandomForestClassifier(n_estimators=100,
                             random_state=1,
                             n_jobs=-1,
                             min_samples_leaf = 50,
                             oob_score=True)

In [95]:
rfc.fit(train[var].fillna(-1), train[target])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=50, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=True, random_state=1, verbose=0, warm_start=False)

In [96]:
print("Performance on Validation:")
_ = evaluate_model(rfc, valid, var)

Performance on Validation:
PRAUC: 0.174, Precision@20Recall: 0.242, AUC: 0.783


### Train on Subset of Features

Train a new random forest classifier using top 150 most important features to see if there is any improvement

In [18]:
# get featuere importance
feat_imp = get_feature_importance(rfc, var)

In [76]:
# subset top 150 most important features 
feat_subset = feat_imp.head(150)['feature']

In [77]:
# train rf again
rfc.fit(train[feat_subset].fillna(-1), train[target])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=50, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=True, random_state=1, verbose=0, warm_start=False)

In [91]:
print("Performance on Validation:")
_ = evaluate_model(rfc, valid, feat_subset)

Performance on Validation:
PRAUC: 0.174, Precision@20Recall: 0.220, AUC: 0.784


There is no improvement on precision-recall AUC. Continue to use all dependent in the following models

### Fine Tune Random Forest with GridSearchCV

In [48]:
# define parameters grid
param_grid = {
    'max_features': ['auto', 10, 20],
    'min_samples_leaf': [50, 100, 200]
}

In [49]:
base_rf = RandomForestClassifier(n_estimators=100,random_state=1,n_jobs=-1)

In [50]:
rf_best = GridSearchCV(base_rf, param_grid, scoring='average_precision', cv=3)

In [51]:
rf_best.fit(train[var].fillna(-1), train[target])

GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=1, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_features': ['auto', 10, 20], 'min_samples_leaf': [50, 100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='average_precision', verbose=0)

In [71]:
rf_best.best_params_

{'max_features': 20, 'min_samples_leaf': 50}

In [130]:
print("Avg PRAUC of 3-fold CV on train set: %.3f" %rf_best.best_score_)

Avg PRAUC of 3-fold CV on train set: 0.155


In [85]:
print("Performance on Validation:")
_ = evaluate_model(rf_best, valid, var)

Performance on Validation:
PRAUC: 0.183, Precision@20Recall: 0.233, AUC: 0.786


Precision-recall AUC increased from 0.174 to 0.183 on validation set after fine tuning.

### Base Light GBM

In [87]:
# initialize gradient boosting model
gbm = lgb.LGBMClassifier(n_estimators=200,
                         random_state=1,
                         colsample_bytree=0.5,
                         learning_rate=0.05,
                         min_child_samples=50,
                         subsample=0.9,
                         objective='binary')

In [88]:
# fit the model
gbm.fit(train[var].fillna(-1), train[target],
        eval_metric = avg_precision, 
        eval_names = ['train','valid'],
        eval_set=[(train[var], train[target]), (valid[var], valid[target])],
        early_stopping_rounds=20,
        verbose=10)

Training until validation scores don't improve for 20 rounds.
[10]	train's binary_logloss: 0.41846	train's pr_auc: 0.209791	valid's binary_logloss: 0.419287	valid's pr_auc: 0.169956
[20]	train's binary_logloss: 0.299295	train's pr_auc: 0.222562	valid's binary_logloss: 0.301013	valid's pr_auc: 0.174298
[30]	train's binary_logloss: 0.233251	train's pr_auc: 0.236152	valid's binary_logloss: 0.235911	valid's pr_auc: 0.182064
[40]	train's binary_logloss: 0.199083	train's pr_auc: 0.243126	valid's binary_logloss: 0.202561	valid's pr_auc: 0.185409
[50]	train's binary_logloss: 0.183553	train's pr_auc: 0.255036	valid's binary_logloss: 0.188029	valid's pr_auc: 0.18454
Early stopping, best iteration is:
[38]	train's binary_logloss: 0.204573	train's pr_auc: 0.24292	valid's binary_logloss: 0.207895	valid's pr_auc: 0.185413


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.5,
        learning_rate=0.05, max_depth=-1, min_child_samples=50,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=200,
        n_jobs=-1, num_leaves=31, objective='binary', random_state=1,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=0.9,
        subsample_for_bin=200000, subsample_freq=1)

In [89]:
print("Performance on Validation:")
_, auc_train, pr_auc_train, precision_r20_train = evaluate_model(gbm, valid, var)

Performance on Validation:
PRAUC: 0.188, Precision@20Recall: 0.253, AUC: 0.788


Precision-recall AUC increased from 0.183 to 0.188 on validation set using LightGBM.

### Fine Tune LightGBM with GridSearchCV

In [63]:
# initialize base GBM and define parameter grid
base_gbm = lgb.LGBMClassifier(num_leaves=31)

param_grid = {
    'learning_rate': [0.05, 0.1],
    'n_estimators': [50, 100],
    'colsample_bytree': [0.5, 0.8],
    'min_child_samples': [50, 100],
    'subsample': [0.8, 1]
}

best_gbm = GridSearchCV(base_gbm, param_grid, scoring='average_precision', cv=3)

In [64]:
# fit model
best_gbm.fit(train[var].fillna(-1), train[target])

GridSearchCV(cv=3, error_score='raise',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'learning_rate': [0.05, 0.1], 'n_estimators': [50, 100], 'colsample_bytree': [0.5, 0.8], 'min_child_samples': [50, 100], 'subsample': [0.8, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='average_precision', verbose=0)

In [74]:
# show best parameters to use
best_gbm.best_params_

{'colsample_bytree': 0.5,
 'learning_rate': 0.05,
 'min_child_samples': 50,
 'n_estimators': 100,
 'subsample': 1}

In [70]:
print("PRAUC on train 3-fold CV : %.3f" %best_gbm.best_score_)

PRAUC on train 3-fold CV : 0.170


In [90]:
print("Performance on Validation:")
_, auc_train, pr_auc_train, precision_r20_train = evaluate_model(best_gbm, valid, var)

Performance on Validation:
PRAUC: 0.193, Precision@20Recall: 0.253, AUC: 0.795


Precision-recall AUC increased from 0.188 to 0.193 after fine tuning.

### Ensemble

Use the average of predictions from random forest and gbm, check performance

In [131]:
print("Performance on Validation:")
_, auc_train, pr_auc_train, precision_r20_train = evaluate_model([best_gbm, rf_best], valid, var)

Performance on Validation:
PRAUC: 0.193, Precision@20Recall: 0.242, AUC: 0.793


There is no improvement using emsemble

### Final Model

In [97]:
# chose gbm as final model, use optimal parameters found by grid search cv
gbm_final = base_gbm.set_params(**best_gbm.best_params_)

In [98]:
# use full data set (train + validation) to train final model
full_data = pd.concat([train, valid], axis=0)

In [102]:
gbm_final.fit(full_data[var], full_data[target])

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.5,
        learning_rate=0.05, max_depth=-1, min_child_samples=50,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1,
        subsample_for_bin=200000, subsample_freq=1)

In [105]:
# predict on test data
prediction = gbm_final.predict_proba(test[var])[:,1]

In [108]:
# put into data frame
pred_df = pd.DataFrame({"patient_id": test['patient_id'],
                        "dm2_prob"  : prediction})

pred_df = pred_df[['patient_id', 'dm2_prob']]

In [112]:
# write into csv
pred_df.to_csv("./jadeyun_dm2_solution.csv", index=False)

### Understand Feature Importance

In [113]:
rf_final = base_rf.set_params(**rf_best.best_params_)

In [114]:
rf_final.fit(full_data[var].fillna(-1), full_data[target])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=20, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=50, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [116]:
rf_feat_imp = get_feature_importance(rf_final, var)

In [127]:
#rf_feat_imp.head(10).plot.bar(x='feature')  # uncomment for visulization
rf_feat_imp.head(15)

,feature,importance
0,age,0.101332
167,avg_loinc_4548_4,0.061449
30,cnt_icd10_I10,0.030264
98,d_first_icd10_I10,0.029156
78,d_last_icd10_I10,0.027610
68,d_first_visit,0.019600
100,d_first_icd9_401_9,0.018913
69,d_last_visit,0.018240
21,num_visit_12mo,0.018083
164,avg_loinc_27353_2,0.017795


In [123]:
gbm_feat_imp = get_feature_importance(gbm_final, var)

In [128]:
#gbm_feat_imp.head(10).plot.bar(x='feature')
gbm_feat_imp.head(15)

,feature,importance
0,age,176
69,d_last_visit,87
76,d_last_cpt_83036,87
182,v_last_loinc_4548_4,74
167,avg_loinc_4548_4,60
2,num_total_res,52
16,num_visit,52
100,d_first_icd9_401_9,51
106,d_first_icd10_R73_09,48
98,d_first_icd10_I10,47
